In [6]:
#!pip install pandas nltk
#!pip install tqdm

In [1]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter



In [2]:
input_file = "../data/toxic_comments_cleaned.csv"
output_file = "../data/processed.csv"
vocab_file = '../data/vocab.csv'

In [3]:
# Load the cleaned data from CSV
df = pd.read_csv(output_file)
# Load the vocabulary from CSV
vocab = pd.read_csv(vocab_file)
# Turn vocab into map
vocab = dict(zip(vocab['word'], vocab['id']))

C:\Users\msi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Dataset should be downloaded from Here(https://www.kaggle.com/datasets/apoorvyadav/toxiccommentscleaned)

In [9]:
df = pd.read_csv(input_file)

C:\Users\msi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
df.head()

,id,comment_text,target
0,0000997932d777bf,explanation why the edits made under my userna...,0.0
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0.0
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0.0
3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0.0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0.0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11912547 entries, 0 to 11912546
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            object 
 1   comment_text  object 
 2   target        float64
dtypes: float64(1), object(2)
memory usage: 272.7+ MB


In [12]:
(df[df["target"] == 1]["id"].count()) / df["id"].count()

0.0140022112819366

In [13]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\msi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\msi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\msi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
def clean_text(text):
    try:
        if isinstance(text, str):
            # Remove special characters, numbers, and extra spaces
            text = re.sub(r'[^a-zA-Z\s]', '', text)

            # Convert text to lowercase
            text = text.lower()

            # Tokenize text
            words = word_tokenize(text)

            # Remove stopwords
            stop_words = set(stopwords.words('english'))
            words = [word for word in words if word not in stop_words]

            # Apply stemming
            stemmer = PorterStemmer()
            stemmed_words = [stemmer.stem(word) for word in words]

            return ' '.join(stemmed_words)

        return ''  # Return empty string if not a string

    except Exception as e:
        return ''  # Return empty string in case of any error (Memory errors or Buffer overflow)


In [15]:
clean_text(df['comment_text'].iloc[3])


'cant make real suggest improv wonder section statist later subsect type accid think refer may need tidi exact format date format later noon els first prefer format style refer want pleas let know appear backlog articl review guess may delay review turn list relev form exampl wikipediagoodarticlenominationstransport'

In [16]:
df = df[(df['comment_text'].apply(lambda x: len(str(x).split())) <= 200) &
        (df['comment_text'].apply(lambda x: len(str(x)) <= 2000))]

In [17]:
# Load dataset in chunks
chunk_size = 10  # Process 1000 rows at a time

# Read CSV in chunks
chunks = pd.read_csv(input_file, chunksize=chunk_size)

# Enable tqdm progress bar
tqdm.pandas()

# Open output file to store processed data
with open(output_file, 'w', encoding='utf-8') as f_out:
    first_chunk = True  # To write headers only once

    for chunk in tqdm(chunks, desc="Processing chunks"):
        # Apply text cleaning function (NO recursion issues)
        chunk['cleaned_comment'] = chunk['comment_text'].apply(clean_text)

        # Append to CSV (write header only for the first chunk)
        chunk.to_csv(f_out, index=False, header=first_chunk, mode='a', encoding='utf-8')
        first_chunk = False  # Ensure only the first chunk writes headers

Processing chunks: 202878it [49:28, 68.34it/s]


In [18]:
clean_text(df['comment_text'].iloc[115606])


'long feedback'

In [11]:
#Removing empty cleaned text comment
df = df.dropna(how='all')
df = df.dropna(subset=['cleaned_comment'])

In [40]:
df.to_csv(output_file)


C:\Users\msi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Tokenization cleaned comments

In [23]:
nltk.download('punkt', quiet=True)

# Tokenize all comments
df['tokenized_comment'] = df['cleaned_comment'].apply(word_tokenize)
print(df['tokenized_comment'].head())


0    [explan, edit, made, usernam, hardcor, metalli...
1    [daww, match, background, colour, im, seemingl...
2    [hey, man, im, realli, tri, edit, war, guy, co...
3    [cant, make, real, suggest, improv, wonder, se...
4               [sir, hero, chanc, rememb, page, that]
Name: tokenized_comment, dtype: object


In [44]:
import pandas as pd
import ast
from collections import Counter

# Define a function to process a chunk
def process_chunk(chunk):
    chunk['tokenized_comment'] = chunk['tokenized_comment'].apply(ast.literal_eval)
    return chunk

# Define a function to update the vocabulary
def update_vocab(vocab_counter, chunk):
    for sublist in chunk['tokenized_comment']:
        vocab_counter.update(sublist)

# Initialize counter for vocabulary
vocab_counter = Counter()

# Read data in chunks (adjust chunk size as needed)
chunk_size = 1000  # Adjust based on available RAM
df_chunks = pd.read_csv(output_file, chunksize=chunk_size)  # Use chunksize to avoid loading entire file

for chunk in df_chunks:
    chunk = process_chunk(chunk)  # Convert tokenized comments from string to list
    update_vocab(vocab_counter, chunk)  # Update vocabulary counter

# Initialize vocabulary dictionary with PAD to  ken set to ID 0
vocab = {'[PAD]': 0}

# Assign unique IDs to words
vocab.update({word: idx for idx, (word, _) in enumerate(vocab_counter.items(), start=1) if word != '[PAD]'})

print(f"Vocabulary size: {len(vocab)}")


Vocabulary size: 603538


In [48]:
# Saving vocab
vocab_df = pd.DataFrame(list(vocab.items()), columns=['word', 'id'])
vocab_df.to_csv(vocab_file, index=False)

In [50]:
# Load the vocabulary from CSV
vocab = pd.read_csv(vocab_file)
vocab = dict(zip(vocab['word'], vocab['id']))

In [ ]:
# Function to convert tokens to word IDs
def words_to_ids(tokens, vocab):
    return [vocab.get(word) for word in tokens]
# Apply the word-to-ID conversion to the tokenized comments
df['sequence'] = df['tokenized_comment'].apply(lambda tokens: words_to_ids(tokens, vocab))
# Show the DataFrame with the word IDs
print(df[['tokenized_comment', 'sequence']].head())


In [8]:
import pandas as pd
import ast
import os

# Function to convert tokens to word IDs
def words_to_ids(tokens, vocab):
    return [vocab.get(word, 0) for word in tokens]  # Default to 0 if word is not in vocab

# In-place processing function
def process_in_place(file_path, vocab, chunk_size=10000):
    temp_file = file_path + ".tmp"  # Temporary file to store processed data

    with open(temp_file, 'w') as f:
        first_chunk = True  # Track if we are processing the first chunk

        # Process file in chunks
        for chunk in pd.read_csv(file_path, chunksize=chunk_size):
            # Ensure 'tokenized_comment' is read as a list
            chunk['tokenized_comment'] = chunk['tokenized_comment'].apply(ast.literal_eval)

            # Convert words to IDs
            chunk['sequence'] = chunk['tokenized_comment'].apply(lambda tokens: words_to_ids(tokens, vocab))

            # Save processed chunk to temporary file
            chunk.to_csv(f, mode='a', header=first_chunk, index=False)
            first_chunk = False  # Ensure only the first chunk writes the header

    # Replace the original file with the processed file
    os.replace(temp_file, file_path)
    print("Processing completed. File updated in-place.")

# Example usage
process_in_place(output_file, vocab)


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: '../data/processed.csv.tmp' -> '../data/processed.csv'

In [7]:
df.to_csv(output_file, index=False)


In [10]:
df = pd.read_csv(output_file)
df = df.dropna(how='all')
df.to_csv(output_file, index=False)


C:\Users\msi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
max_word_count = max(df['comment_text'].apply(lambda x: len(str(x).split())))
print(max_word_count)


3750


In [32]:
max_index = df['comment_text'].apply(lambda x: len(str(x).split())).idxmax()
print(df.loc[max_index, 'comment_text'])  # Prints the comment with max words


laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing out loud laughing o